#### Imports packages needed and defines GLOBAL variables

In [1]:
# All imports needed
from ultralytics import YOLO
import os
import cv2 as cv
import numpy as np
from PIL import Image
import csv

tmp_crop = "tmp\\crop.png"
tmp_mask = "tmp\\mask.png"

### Loads pre-trained models

In [2]:
cropResistor = YOLO("D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\resistor-wizard\\Backend\\YOLO_Cropper\\latest\\best.pt")
findColorBands = YOLO("D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\resistor-wizard\\Backend\\YOLO_Segmenter\\latest\\best.pt")

### Functions used for handling inferences

In [3]:
# Crops a numpy array image using a bounding box
def cropImage(image, box):
    return image[int(box[1]):int(box[3]), int(box[0]):int(box[2])]

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

# Checks if two colors are equal
def eqq(a,b):
    if len(a) != len(b):
        return False
    for i in range(len(a)):
        if a[i] != b[i]:
                return False
    return True

# Class mask for grouping objects
class Mask:
    def __init__(self, img, bbox=[], contour=[]):
        self.image = img
        self.orig_shape = img.shape
        self.mask = np.zeros_like(img, dtype=np.uint8)  # creates mask of same size as original image
        self.size = -1  # Number of non-zero pixels
        self.avgColor = (None, None, None)
        self.bbox = bbox
        self.contour = contour
        self.index = -1  # This color's position relative to others
        self.color = ""  # A string identifier of the color the mask represents
    # Bitwise AND on the original image using the mask
    def maskImage(self):
        pixels = np.zeros_like(self.image, dtype=np.uint8)
        for x in range(len(self.image)):
            for y in range(len(self.image[0])):
                if eqq(self.mask[x][y], [255,255,255]):
                    pixels[x][y] = self.image[x][y]
        return pixels
    # Calculates average color on the color band associated with the mask
    def sample_avg_color(self):
        pixels = self.maskImage()
        self.avgColor = np.sum(pixels.reshape((-1,1,3)), axis=0) / self.size   
    # Checks if a point is inside the mask's bounding box
    def contains(self, point):
        if list(point) < self.bbox[0:2].tolist() or list(point) > self.bbox[2:4].tolist():
            return False
        else:
            return True

# Calculates the centroid of a rectangle
def get_centroid(vertex):
    cX = (vertex[2] + vertex[0])/2
    cY = (vertex[3] + vertex[1])/2
    return (cX, cY)

# Calculates the linear regression of the points in a 2D dataset: Y in function of X (Y= a*X + b)
def lin_reg(data):
    X = np.array([i[0] for i in data])
    Y = np.array([i[1] for i in data])
    a = ((len(data) * np.sum(X*Y)) - (np.sum(X) * np.sum(Y)))/(len(data) * np.sum(X*X) - (np.sum(X))**2)
    b = ((np.sum(Y) * np.sum(X*X)) - (np.sum(X) * np.sum(X*Y)))/(len(data) * np.sum(X*X) - (np.sum(X))**2)
    return a,b

def cvtBGR2HSV(bgr, paint=False):
    B,G,R = bgr
    Rp = R/255 
    Gp = G/255 
    Bp = B/255
    Cmax = max(Rp, Gp, Bp)
    Cmin = min(Rp, Gp, Bp)
    delta = Cmax - Cmin
    #
    if delta == 0:
        H = 0
    elif Cmax == Rp:
        H = 60 * (((Gp - Bp) / delta) % 6)
    elif Cmax == Gp:
        H = 60 * (((Bp - Rp) / delta) + 2)
    elif Cmax == Bp:
        H = 60 * (((Rp - Gp) / delta) + 4)
    #
    if Cmax == 0:
        S = 0
    else:
        S = delta/Cmax
    #
    V = Cmax

    if paint:
        S *= 100
        V *= 100
    else:
        S *= 255
        V *= 255
    return [int(i) for i in (H,S,V)]
    

# Retrieves the pixels contained in the contours given by the model prediction
def get_segmentation_masks(img, inference, data=255):
    image = cv.imread(img)
    masks = [Mask(image) for i in range(len(inference[0].masks.xy))]
    #
    for i in range(len(inference[0].masks.xy)):
        polygon = inference[0].masks.xy[i].astype(int)
        masks[i].bbox = inference[0].boxes.cpu().data[i]
        masks[i].contour = polygon.reshape((-1, 1, 2))  # Reshape to match the format expected by cv.drawContours
        masks[i].mask = cv.fillPoly(masks[i].mask, [masks[i].contour], (data, data, data)) # converts polygon points to pixels inside it
        masks[i].size = 0
        for j in masks[i].mask.reshape((-1,1,3)):
            if not eqq(j[0], [0,0,0]):
                masks[i].size += 1
        masks[i].sample_avg_color()
    return masks

# Finds the mask that contains a given point – considers a point can belong to a single mask
def retrieve_point_in_mask(masks, point):
    for mask in masks:
        if mask.contains(point):
            return mask

# Calculates the possible orders of the color bands – assumes the bands are approximately
#     linear, which is reasonable given that resistors follow this standard
def order_masks(masks, inference):
    boxes = inference[0].boxes.cpu().data
    vertex = [v[:4] for v in boxes]
    centroids = [get_centroid(v) for v in vertex]
    #
    # Calculates the line that approximately crosses the centroids – it's used to find the order in which the bands are disposed
    parameters = lin_reg(centroids)
    lv = np.array([1,parameters[0]]) # Line Vector
    lv_norm = np.sqrt(sum(lv**2))
    #
    vects = {}
    for center in centroids:
        u = np.array(center) - np.array([0,parameters[1]])  # Calculates the vector from the origin of the line to the centroid
        proj = lv * np.dot(u,lv) / (lv_norm**2)  # Projects the centroid onto the line
        vects[retrieve_point_in_mask(masks, center)] = np.sqrt(np.sum(proj**2))  # Calculates each centroid's distance to origin and associates it to its respective mask
    # Sorts masks by distance
    sorted_masks = dict(sorted(vects.items(), key=lambda item: item[1]))
    return list(sorted_masks)


# Converts HSV values to paint's standard
def cvtHSV(hsv):
    return [int(i) for i in [hsv[0]*2, 0.3922*hsv[1], 0.3922*hsv[2]]]


### Code used for extracting and handling color classes

In [4]:
def extract_colors(directory):
    classes = dict()
    for file in os.listdir(directory):
        if os.path.isfile(file):
            ext = file.split('.')[-1]
            name = ".".join(file.split('.')[:-1])
            if ext == "csv":
                with open(file) as csv_file:
                    csv_reader = csv.reader(csv_file, delimiter=',')
                    line_count = 0
                    summ = np.array([0,0,0])
                    for row in csv_reader:
                        H = int(row[0])
                        S = int(row[1])
                        V = int(row[2])
                        hsv = np.array([H,S,V])
                        line_count += 1
                        summ += hsv
                
                classes[name] = summ/line_count
    return classes

### Merged inference code

In [76]:
file = "D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\resistor-wizard\\Backend\\Tests\\huechange.jpg"

In [71]:
# Runs crop detection model on file
cropped_inf = cropResistor(file, conf=0.6)

# Crops and saves original file to a temporary location
if len(cropped_inf[0].boxes.data):
    cropped = cropImage(cv.imread(file), cropped_inf[0].boxes.cpu().data[0])
else:
    cropped = cv.imread(file)
Image.fromarray(cv.cvtColor(cropped, cv.COLOR_BGR2RGB)).save(tmp_crop)


image 1/1 D:\Joule\Documents\Jhony\Universidade\UTFPR\2023.2\Oficinas\resistor-wizard\Backend\Tests\WhatsApp Image 2023-10-01 at 19.33.39_581eccb2 - Copy.jpg: 288x640 (no detections), 169.3ms
Speed: 3.0ms preprocess, 169.3ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 640)


In [77]:
# Runs color bands segmentation model on cropped inference
if len(cropped_inf[0].boxes.data):
    colorbands_inf = findColorBands(tmp_crop, save=True, conf=0.6, iou=0.3)
else:
    colorbands_inf = findColorBands(file, save=True, conf=0.6, iou=0.3)


image 1/1 D:\Joule\Documents\Jhony\Universidade\UTFPR\2023.2\Oficinas\resistor-wizard\Backend\Tests\huechange.jpg: 288x640 1 Resistor-Color-Bands, 1 Resistor-Color-Bands-Start, 11.1ms
Speed: 1.0ms preprocess, 11.1ms inference, 3.5ms postprocess per image at shape (1, 3, 288, 640)
Results saved to runs\segment\predict3


### Finds the order of bands

In [73]:
# Gets masks objects from the inference
masks = get_segmentation_masks(tmp_crop, colorbands_inf)

# Sorts the masks by distance (it doesn't have a way to 
#    know in which direction to start, so by default it 
#    uses the leftmost (0,0) point as a beginning.
ordered = order_masks(masks, colorbands_inf)

AttributeError: 'NoneType' object has no attribute 'xy'

In [74]:
for i in range(len(ordered)):
    print(f"BBOX: {ordered[i].bbox[:4]} \t\t HSV: {cvtBGR2HSV(ordered[i].avgColor[0], paint=True)}")

BBOX: tensor([3.5735e+01, 2.1792e-02, 5.7082e+01, 5.9894e+01]) 		 HSV: [229, 25, 47]
BBOX: tensor([65.6920,  8.9955, 82.6714, 55.9359]) 		 HSV: [224, 28, 51]
BBOX: tensor([ 90.1763,  33.7596, 103.1645,  58.3405]) 		 HSV: [212, 56, 45]
BBOX: tensor([111.5066,  14.7869, 127.2329,  61.4750]) 		 HSV: [231, 50, 67]
BBOX: tensor([134.1161,  13.2323, 149.3516,  71.3570]) 		 HSV: [109, 32, 66]


In [75]:
# Retrieves the color name from masks average color
# Numbers on the side represent the number of samples used for each color's average
#  Files used to sample: res2.png, res.png, crop.jpg
COLOR_RANGES = {}
def sample_color_names(mask):
    pass